In [114]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re

%matplotlib inline
pd.set_option('display.width', 1500)
pd.set_option('display.max_columns', 100)

In [115]:
# Install read xlsx package
! conda install --yes xlrd 

Solving environment: done

# All requested packages already installed.



In [116]:
base_df = pd.read_csv('data/usa_2016_newdf.csv')

In [117]:
base_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3143 entries, 0 to 3142
Columns: 159 entries, State to Dem08 Frac2
dtypes: float64(147), int64(2), object(10)
memory usage: 3.8+ MB


In [118]:
## Change Fips into string for merge
base_df.dtypes
base_df['Fips'] = base_df['Fips'].astype(str)

In [119]:
base_df.head()

,State,ST,Fips,County,Precincts,Votes,Democrats 08 (Votes),Democrats 12 (Votes),Republicans 08 (Votes),Republicans 12 (Votes),Republicans 2016,Democrats 2016,Green 2016,Libertarians 2016,Republicans 2012,Republicans 2008,Democrats 2012,Democrats 2008,Less Than High School Diploma,At Least High School Diploma,At Least Bachelors's Degree,Graduate Degree,School Enrollment,Median Earnings 2010,White (Not Latino) Population,African American Population,Native American Population,Asian American Population,Other Race or Races,Latino Population,Children Under 6 Living in Poverty,Adults 65 and Older Living in Poverty,Total Population,Preschool.Enrollment.Ratio.enrolled.ages.3.and.4,Poverty.Rate.below.federal.poverty.threshold,Gini.Coefficient,Child.Poverty.living.in.families.below.the.poverty.line,Management.professional.and.related.occupations,Service.occupations,Sales.and.office.occupations,Farming.fishing.and.forestry.occupations,Construction.extraction.maintenance.and.repair.occupations,Production.transportation.and.material.moving.occupations,White,Black,Hispanic,Asian,Amerindian,Other,White Asian,...,reporting,Votes16 Trumpd,Votes16 Clintonh,Votes16 Johnsong,Votes16 Steinj,Votes16 Castled,Votes16 De La Fuenter,Est Votes Remaining,Votes16 Mcmulline,Votes16 Hedgesj,Votes16 Kahnl,Votes16 La Rivag,Votes16 Hoeflingt,Votes16 Kenistonc,Votes16 Smithm,Votes16 Atwoodf,Votes16 Kennedya,Votes16 Kopitkek,Votes16 Kotlikoffl,Votes16 Lyttleb,Votes16 Maldonadoj,Votes16 Maturenm,Votes16 Scottr,Votes16 Silvar,Votes16 Soltysike,Votes16 Vacekd,Votes16 Copelands,Votes16 Jacobp,Votes16 Whitej,Votes16 Mooreheadm,Votes16 None Of These Candidates,Votes16 Duncanr,Votes16 Skewesp,Votes16 Giordanir,total16,Other16 Frac,Rep16 Frac2,Dem16 Frac2,Name Prev,Statecode Prev,total08,total12,other08,other12,Other12 Frac,Other08 Frac,Rep12 Frac2,Rep08 Frac2,Dem12 Frac2,Dem08 Frac2
0,Georgia,GA,13089,"DeKalb County, Georgia",192.0,295871.0,254594.0,238224.0,65581.0,64392.0,16.064771,80.822723,NaN,3.112505,21.024929,20.347560,77.783619,78.991883,12.1,87.9,38.7,15.1,76.15,31275.05492,30.05,53.60,0.20,4.6,1.65,9.95,25.75,10.75,712476,56.9,15.75,0.472,24.00,41.30,15.10,25.45,0.05,8.15,9.85,30.05,53.60,9.95,4.6,0.20,1.65,34.65,...,192.0,47531.0,239131.0,9209.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,295871.0,0.031125,0.165809,0.834191,DeKalb,13.0,322304.0,306265.0,2129.0,3649.0,0.011915,0.006606,0.212785,0.204829,0.787215,0.795171
1,Texas,TX,48487,"Wilbarger County, Texas",7.0,4101.0,1196.0,971.0,3283.0,2956.0,77.200683,19.678127,0.316996,2.804194,74.271357,72.809936,24.396985,26.524728,26.4,73.6,15.9,5.9,69.25,22438.66508,63.30,8.90,0.70,0.4,1.10,25.55,52.95,11.60,13630,32.4,24.30,0.454,37.05,26.25,25.50,18.10,2.35,12.95,14.75,63.30,8.90,25.55,0.4,0.70,1.10,63.70,...,7.0,3166.0,807.0,115.0,13.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4101.0,0.028042,0.796879,0.203121,Wilbarger,48.0,4509.0,3980.0,30.0,53.0,0.013317,0.006653,0.752737,0.732976,0.247263,0.267024
2,Virginia,VA,51111,"Lunenburg County, Virginia",12.0,5577.0,2703.0,2684.0,2900.0,2969.0,57.486104,39.913932,0.448270,1.577909,51.778863,51.327434,46.808511,47.840708,28.5,71.5,10.0,3.0,72.35,24528.98095,59.30,35.70,0.15,0.1,1.45,3.35,32.35,16.95,12911,35.4,18.55,0.392,27.35,21.15,19.65,26.75,1.90,10.75,19.85,59.30,35.70,3.35,0.1,0.15,1.45,59.40,...,12.0,3206.0,2226.0,88.0,25.0,NaN,NaN,0.0,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5577.0,0.015779,0.590206,0.409794,Lunenburg,51.0,5650.0,5734.0,47.0,81.0,0.014126,0.008319,0.525208,0.517580,0.474792,0.482420
3,Georgia,GA,13297,"Walton County, Georgia",21.0,40425.0,8469.0,8148.0,27253.0,29036.0,76.915275,20.479901,NaN,2.604824,77.198766,75.648143,21.663299,23.508022,19.6,80.4,17.3,5.3,73.75,29155.95143,78.90,15.45,0.30,1.1,1.00,3.10,22.25,8.85,83005,44.2,12.

In [120]:
unemploy16 = pd.read_excel('data/unemploy16.xlsx', converters={'County FIPS':str})
# Change State FIPS Code into string
unemploy16['State FIPS Code'] = unemploy16['State FIPS Code'].astype(str)

In [121]:
# merge State Fips into County Fips
unemploy16['Fips'] = unemploy16['State FIPS Code'] + unemploy16['County FIPS']
unemploy16.head()
#update my new unemployment dataframe
unemploy16_sel = unemploy16[['Fips', 'County Name/State Abbreviation', 'Unemployment Rate (%)']]

In [122]:
unemploy16_sel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3219 entries, 0 to 3218
Data columns (total 3 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Fips                            3219 non-null   object 
 1   County Name/State Abbreviation  3219 non-null   object 
 2   Unemployment Rate (%)           3219 non-null   float64
dtypes: float64(1), object(2)
memory usage: 75.6+ KB


In [123]:
base_df['Fips'] = base_df['Fips'].astype(int)
base_df = base_df.sort_values('Fips')
base_df['Fips'] = base_df['Fips'].astype(str)

In [124]:
# extract first counties names
base_df['County'] = base_df['County'].str.extract(r'(\w*\s*\w*),')

In [125]:
base_df_updated = base_df[['Fips', 'State', 'County']].reset_index(drop=True)

In [126]:
base_unemploy = pd.merge(base_df_updated, unemploy16_sel, how='left', on=['Fips'])

In [127]:
## Select the columns that I'm interested
base_unemploy = base_unemploy[['Fips', 'State', 'County', 'Unemployment Rate (%)']]

In [128]:
# Here are the data that is missing total there are 4 that is missing unemployment rate
base_unemploy[base_unemploy['Unemployment Rate (%)'].isnull()]

,Fips,State,County,Unemployment Rate (%)
92,2270,Alaska,Census Area,NaN
548,15005,Hawaii,Kalawao County,NaN
2417,46113,South Dakota,Shannon County,NaN
2916,51515,Virginia,Bedford city,NaN


In [129]:
base_unemploy.info() # This includes 4 enties that have "NaN" for unemployment Rate

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3143 entries, 0 to 3142
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Fips                   3143 non-null   object 
 1   State                  3143 non-null   object 
 2   County                 3143 non-null   object 
 3   Unemployment Rate (%)  3139 non-null   float64
dtypes: float64(1), object(3)
memory usage: 122.8+ KB


In [130]:
# make a copy of our base_unemploy
base_unemploy_train = base_unemploy.copy()
base_unemploy_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3143 entries, 0 to 3142
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Fips                   3143 non-null   object 
 1   State                  3143 non-null   object 
 2   County                 3143 non-null   object 
 3   Unemployment Rate (%)  3139 non-null   float64
dtypes: float64(1), object(3)
memory usage: 122.8+ KB


In [131]:
# combine response varaible to the training model
presidential_result = pd.read_csv('data/presidential_election_returns_2012_2016.csv')
presidential_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12632 entries, 0 to 12631
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            12632 non-null  int64  
 1   state           12632 non-null  object 
 2   state_po        12616 non-null  object 
 3   county          12632 non-null  object 
 4   FIPS            12620 non-null  float64
 5   office          12632 non-null  object 
 6   candidate       12632 non-null  object 
 7   party           12632 non-null  object 
 8   candidatevotes  12624 non-null  float64
 9   totalvotes      12632 non-null  int64  
 10  version         12632 non-null  int64  
 11  result          12624 non-null  float64
dtypes: float64(3), int64(3), object(6)
memory usage: 1.2+ MB


In [132]:
## Get the ones that are only 2016

pres_df_2016 = presidential_result[presidential_result.year==2016]
pres_df_2016[pres_df_2016['FIPS'].isnull()] 

# There are some FIPS that are NaN
# Theses counties do not appear on our base_unemploy_train; so we are going to drop them 

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version,result
12462,2016,Connecticut,NaN,Statewide writein,NaN,President,Hillary Clinton,democrat,NaN,5056,20191203,NaN
12463,2016,Maine,NaN,Maine UOCAVA,NaN,President,Hillary Clinton,democrat,3017.0,5056,20191203,0.596717
12464,2016,Rhode Island,NaN,Federal Precinct,NaN,President,Hillary Clinton,democrat,637.0,5056,20191203,0.125989
12465,2016,Connecticut,NaN,Statewide writein,NaN,President,Donald Trump,republican,NaN,5056,20191203,NaN
12466,2016,Maine,NaN,Maine UOCAVA,NaN,President,Donald Trump,republican,648.0,5056,20191203,0.128165
12467,2016,Rhode Island,NaN,Federal Precinct,NaN,President,Donald Trump,republican,53.0,5056,20191203,0.010483


In [133]:
# Here is the example for Rhode Island
base_unemploy_train[base_unemploy_train['State']=='Rhode Island'] 

,Fips,State,County,Unemployment Rate (%)
2311,44001,Rhode Island,Bristol County,4.6
2312,44003,Rhode Island,Kent County,4.7
2313,44005,Rhode Island,Newport County,4.7
2314,44007,Rhode Island,Providence County,5.5
2315,44009,Rhode Island,Washington County,4.7


In [134]:
# drop NaN
pres_df_2016 = pres_df_2016.dropna(subset=['FIPS'])
pres_df_2016[pres_df_2016['FIPS'].isnull()] 

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version,result


In [135]:
pres_df_2016.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6310 entries, 6234 to 12631
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            6310 non-null   int64  
 1   state           6310 non-null   object 
 2   state_po        6308 non-null   object 
 3   county          6310 non-null   object 
 4   FIPS            6310 non-null   float64
 5   office          6310 non-null   object 
 6   candidate       6310 non-null   object 
 7   party           6310 non-null   object 
 8   candidatevotes  6308 non-null   float64
 9   totalvotes      6310 non-null   int64  
 10  version         6310 non-null   int64  
 11  result          6308 non-null   float64
dtypes: float64(3), int64(3), object(6)
memory usage: 640.9+ KB


In [136]:
# covert 'FIPS' to string
pres_df_2016['FIPS'] = pres_df_2016['FIPS'].astype(int)
base_unemploy_train['FIPS'] = base_unemploy_train['Fips'].astype(str)

In [137]:
# change FIPS in pres_df_2016 into object
pres_df_2016['FIPS'] = pres_df_2016['FIPS'].astype(str)

In [138]:
# Check if our two dataframes are okay to be merged
base_unemploy_train

,Fips,State,County,Unemployment Rate (%),FIPS
0,1001,Alabama,Autauga County,5.1,1001
1,1003,Alabama,Baldwin County,5.3,1003
2,1005,Alabama,Barbour County,8.3,1005
3,1007,Alabama,Bibb County,6.4,1007
4,1009,Alabama,Blount County,5.4,1009
...,...,...,...,...,...
3138,56037,Wyoming,Sweetwater County,6.1,56037
3139,56039,Wyoming,Teton County,3.5,56039
3140,56041,Wyoming,Uinta County,5.8,56041
3141,56043,Wyoming,Washakie County,4.8,56043


In [139]:
pres_df_2016.FIPS.value_counts()
pres_df_2016[pres_df_2016.FIPS=='55131']

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version,result
12404,2016,Wisconsin,WI,Washington,55131,President,Hillary Clinton,democrat,20855.0,78003,20191203,0.267362
12405,2016,Wisconsin,WI,Washington,55131,President,Donald Trump,republican,51729.0,78003,20191203,0.663167


In [140]:
# make the merge
pres_train_df = pd.merge(base_unemploy_train, pres_df_2016, how='left', on=['FIPS'])

In [141]:
pres_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6259 entries, 0 to 6258
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Fips                   6259 non-null   object 
 1   State                  6259 non-null   object 
 2   County                 6259 non-null   object 
 3   Unemployment Rate (%)  6253 non-null   float64
 4   FIPS                   6259 non-null   object 
 5   year                   6232 non-null   float64
 6   state                  6232 non-null   object 
 7   state_po               6232 non-null   object 
 8   county                 6232 non-null   object 
 9   office                 6232 non-null   object 
 10  candidate              6232 non-null   object 
 11  party                  6232 non-null   object 
 12  candidatevotes         6230 non-null   float64
 13  totalvotes             6232 non-null   float64
 14  version                6232 non-null   float64
 15  resu

In [142]:
pres_train_df.head()

,Fips,State,County,Unemployment Rate (%),FIPS,year,state,state_po,county,office,candidate,party,candidatevotes,totalvotes,version,result
0,1001,Alabama,Autauga County,5.1,1001,2016.0,Alabama,AL,Autauga,President,Hillary Clinton,democrat,5936.0,24973.0,20191203.0,0.237697
1,1001,Alabama,Autauga County,5.1,1001,2016.0,Alabama,AL,Autauga,President,Donald Trump,republican,18172.0,24973.0,20191203.0,0.727666
2,1003,Alabama,Baldwin County,5.3,1003,2016.0,Alabama,AL,Baldwin,President,Hillary Clinton,democrat,18458.0,95215.0,20191203.0,0.193856
3,1003,Alabama,Baldwin County,5.3,1003,2016.0,Alabama,AL,Baldwin,President,Donald Trump,republican,72883.0,95215.0,20191203.0,0.765457
4,1005,Alabama,Barbour County,8.3,1005,2016.0,Alabama,AL,Barbour,President,Hillary Clinton,democrat,4871.0,10469.0,20191203.0,0.465278


In [143]:
## To double check the rows of our dataframe
len(pres_train_df)

6259

In [144]:
## select only predictors and response variables
#train_df1 = pres_train_df[['FIPS', 'State', 'County', 'Employed', 'Unemployed', 'Unemployment Rate (%)', 
 #                                     'candidate', 'party']]

In [146]:
train_df1 = pres_train_df.rename(columns={'Unemployment Rate (%)': 'Unemploy Rate(%)', 
                                   'candidate': 'PresCandidate', 
                                   'party': 'PresParty'})

In [ ]:
# check to see NaN value for Candidate
train_df1[train_df.PresCandidate.isnull()].head()

In [ ]:
# check to see NaN value for Party:
train_df1[train_df.PresParty.isnull()].head()

In [ ]:
## Seems that there are some candidate from Alaska that are missing candidate votes

In [ ]:
senate_df = pd.read_csv('data/senate_election_returns_2016.csv')
senate_df

In [ ]:
train_df1[train_df1.State == 'Wisconsin'].head()

In [ ]:
senate_df[senate_df.state=='WI']

In [ ]:
# covert fipscode into valid str
#senate_df['fipscode'] = 
senate_df['fipscode'] = senate_df['fipscode'].astype(str).str[:-7]
senate_df = senate_df.rename(columns={'fipscode':'FIPS'})
senate_df 

In [ ]:
senate_df[senate_df.state=='AL']

In [ ]:
## This is our training set with unemployment data

In [ ]:
base_unemploy_train = base_unemploy_train.drop('FIPS', axis=1)

In [ ]:
base_unemploy_train ##### USE THIS FOR further MERGE

In [ ]:
## Here I am creating unemploy Test set

In [ ]:
unemploy20 = pd.read_excel('data/unemploy2020.xlsx')

In [ ]:
unemploy20.info()

In [ ]:
base_df_updated

In [147]:
total_2016 = pd.read_csv("data/totals-2020-11-12T23_42_37 (1).csv")

In [148]:
# This makes sure all are presidents
total_2016.office.value_counts()

P    1770
Name: office, dtype: int64

In [149]:
total_2016.party_full.value_counts

<bound method IndexOpsMixin.value_counts of 0             REPUBLICAN PARTY
1                  INDEPENDENT
2             REPUBLICAN PARTY
3             DEMOCRATIC PARTY
4                        OTHER
                 ...          
1765          DEMOCRATIC PARTY
1766                     OTHER
1767    SOCIALIST PARTY U.S.A.
1768          DEMOCRATIC PARTY
1769               INDEPENDENT
Name: party_full, Length: 1770, dtype: object>

In [150]:
## Split data on both republican democratic party
total_2016_rep = total_2016[total_2016.party_full=='REPUBLICAN PARTY']
total_2016_dem = total_2016[total_2016.party_full=='DEMOCRATIC PARTY']

In [151]:
#display(total_2016_rep.head())
#display(total_2016_dem.head())
predictors = ['receipts', 'disbursements', 'cash_on_hand_end_period']
state_rep = total_2016_rep.groupby('address_state')[predictors].sum()
state_dem = total_2016_dem.groupby('address_state')[predictors].sum()

# print(len(state_rep)) return 47
# print(len(state_dem)) return 47

In [156]:
# rename our columns:
rename_rep = {'receipts':'rep_receipts', 'disbursements':'rep_disburse', 'cash_on_hand_end_period':'rep_cash_on_hand'}
rename_dem = {'receipts':'dem_receipts', 'disbursements':'dem_disburse', 'cash_on_hand_end_period':'dem_cash_on_hand'}

state_rep.rename(columns=rename_rep, inplace=True)
state_dem.rename(columns=rename_dem, inplace=True)

pres_funds_state_2016 = state_rep.merge(state_dem, left_index=True, right_index=True)

In [157]:
pres_funds_state_2016.head()

,rep_receipts,rep_disburse,rep_cash_on_hand,dem_receipts,dem_disburse,dem_cash_on_hand
address_state,,,,,,
AK,0.00,96.0,103.00,0.00,0.0,0.00
AL,205.00,5271.0,0.00,0.00,0.0,0.00
AR,4335975.38,4330417.8,5557.58,0.00,0.0,0.00
AZ,0.00,0.0,0.00,0.00,0.0,0.00
CA,0.00,0.0,0.00,8075989.73,8140359.6,-7718.24


In [160]:
pres_funds_state_2016.to_csv(r'~/Desktop/pres_funds_state_2016.csv')

In [161]:
total_2020 = pd.read_csv("data/totals-2020-11-12T23_58_29.csv")

In [165]:
total_2020.head()

,name,office,office_full,party,party_full,state,district,district_number,election_districts,election_years,cycles,candidate_status,incumbent_challenge,incumbent_challenge_full,load_date,first_file_date,last_file_date,last_f2_date,candidate_id,two_year_period,candidate_election_year,address_city,address_state,address_street_1,address_street_2,address_zip,candidate_inactive,active_through,candidate_id.1,election_year,cycle,is_election,receipts,disbursements,cash_on_hand_end_period,debts_owed_by_committee,coverage_start_date,coverage_end_date,federal_funds_flag,has_raised_funds,party.1,office.1,candidate_inactive.1
0,"ANTHONY, NAKIA LACQUERS",P,President,DEM,DEMOCRATIC PARTY,US,0,0,{00},{2020},"{2018,2020}",C,C,Challenger,2020-02-27 22:00:38,2017-09-05,2017-09-05,2017-09-05,P00006296,2020,2020,YOUNGSTOWN,OH,525 WEST DELASON AVENUE,NaN,44511.0,f,2020,P00006296,2020,2020,t,6363.00,6363.00,0.00,0.00,2019-07-01 00:00:00,2020-10-01 00:00:00,f,t,DEM,P,f
1,"LEFFERT, AKIVA",P,President,DEM,DEMOCRATIC PARTY,US,0,0,{00},{2020},{2020},N,C,Challenger,2019-03-01 21:11:59,2019-02-14,2019-02-14,2019-02-14,P00010108,2020,2020,BROOKLYN,NY,461 15TH STREET,APT 3,11215.0,f,2020,P00010108,2020,2020,t,1073.83,94.74,979.09,0.00,2019-01-01 00:00:00,2020-03-31 00:00:00,f,t,DEM,P,f
2,"OJEDA, RICHARD NEECE II",P,President,DEM,DEMOCRATIC PARTY,US,0,0,{00},{2020},"{2018,2020}",C,C,Challenger,2019-04-03 21:06:26,2018-11-11,2018-11-11,2018-11-11,P00008763,2020,2020,WASHINGTON,DC,1300 I STREET NW,SUITE 400E,20005.0,f,2020,P00008763,2020,2020,t,119477.74,119477.74,0.00,42401.69,2018-10-01 00:00:00,2020-10-14 00:00:00,f,t,DEM,P,f
3,"LEWIS, BILLY R MR III",P,President,AIP,AMERICAN INDEPENDENT PARTY,US,0,0,{00},{2020},{2020},N,C,Challenger,2020-08-21 01:04:24,2020-06-06,2020-06-06,2020-06-06,P00016204,2020,2020,PHILADELPHIA,PA,6608 COTTAGE STREET,NaN,19135.0,f,2020,P00016204,2020,2020,t,0.00,0.00,0.00,0.00,NaN,NaN,f,f,AIP,P,f
4,"MCGEE, ROBERT DERK",P,President,REP,REPUBLICAN PARTY,US,0,0,"{00,00}","{2016,2020}","{2016,2018,2020}",N,C,Challenger,2020-08-21 01:04:24,2015-09-21,2020-08-24,2020-06-26,P60015591,2020,2020,ANGIER,NC,378 WILLIS POPE RD,NaN,27501.0,f,2020,P60015591,2020,2020,t,0.00,0.00,0.00,0.00,NaN,NaN,f,f,REP,P,f


In [166]:
total_2020_rep = total_2020[total_2020['party'] =='REP']
total_2020_dem = total_2020[total_2020['party']=='DEM']

state_rep = total_2020_rep.groupby('address_state')[predictors].sum()
state_dem = total_2020_dem.groupby('address_state')[predictors].sum()

state_rep.rename(columns=rename_rep, inplace=True)
state_dem.rename(columns=rename_dem, inplace=True)

pres_funds_state_2020 = state_rep.merge(state_dem, left_index=True, right_index=True)

In [169]:
pres_funds_state_2020.to_csv(r'~/Desktop/pres_funds_state_2020.csv')